In [ ]:
## Convert the matrix generated by HiC-Pro to the input matrix of ReplicateQC
import pandas as pd
import sys
matrix_inpf = sys.argv[1]
bin_inpf = sys.argv[2]
matrix_outf = sys.argv[3]
bin_outf = sys.argv[4]

df_mtx = pd.read_csv(matrix_inpf, sep="\t", names=['bin1', 'bin2', 'freq'])
df_bin = pd.read_csv(bin_inpf, sep="\t", names=['c', 's', 'e', 'tags'])

## chr tags
def chr_tags_max_min(df_bin):
    #all_chr = df_bin['c'].drop_duplicates(keep="first").tolist()
    df_chr_tags_max = df_bin.groupby(by=['c'], as_index=False)['tags'].max()
    df_chr_tags_min = df_bin.groupby(by=['c'], as_index=False)['tags'].min()
    df_chr_tags_min.columns = ['c', 'min']
    df_chr_tags_max.columns = ['c', 'max']
    df_return = pd.merge(df_chr_tags_min, df_chr_tags_max, how='inner', on=['c'])
    return df_return

## matrix single
def adjust_matrix(df_bin, df_matrix):
    df_chrs_max_min = chr_tags_max_min(df_bin)
    df_matrix_result = pd.DataFrame()
    df_bin_result = pd.DataFrame()
    for index, row in df_chrs_max_min.iterrows():
        start, end = row['min'], row['max']
        df_bin_chr = df_bin.query("@start<=tags<=@end").copy()
        ## matrix
        df_matrix_chr = df_matrix.query("@start<=bin1<=@end & @start<=bin1<=@end").copy()
        df_bin_chr_tags = df_bin_chr[['c', 's', 'tags']].copy()
        df_bin_chr_tags.columns = ['c1', 's1', 'bin1']
        df_one = pd.merge(df_matrix_chr, df_bin_chr_tags, how='left', on=['bin1'])
        df_bin_chr_tags.columns = ['c2', 's2', 'bin2']
        df_two = pd.merge(df_one, df_bin_chr_tags, how='left', on=['bin2'])
        df_matrix_chr_result = df_two[['c1', 's1', 'c2', 's2', 'freq']].copy()
        df_matrix_chr_result.dropna(inplace=True)
        df_matrix_chr_result['s1'] = df_matrix_chr_result['s1'].astype(int)
        df_matrix_chr_result['s2'] = df_matrix_chr_result['s2'].astype(int)
        if df_matrix_result.shape[0] == 0 :
            df_matrix_result = df_matrix_chr_result.copy()
        else:
            df_matrix_result = pd.concat([df_matrix_result, df_matrix_chr_result], axis=0)
        ## bin
        bin_min = df_bin_chr['s'].min()
        df_bin_chr['s'] = df_bin_chr['s'] - bin_min
        df_bin_chr['e'] = df_bin_chr['e'] - bin_min
        df_bin_chr_result = df_bin_chr[['c', 's', 'e', 's']]
        if df_bin_result.shape[0]==0:
            df_bin_result = df_bin_chr_result.copy()
        else:
            df_bin_result = pd.concat([df_bin_result, df_bin_chr_result], axis=0)
    return df_matrix_result, df_bin_result
df_matrix_out, df_bin_out = adjust_matrix(df_bin, df_mtx)
df_matrix_out.to_csv(matrix_outf, sep="\t", index=False, header=False)
df_bin_out.to_csv(bin_outf, sep="\t", index=False, header=False)        